In [3]:
import pandas as pd
data = pd.read_csv("thanksgiving.csv", encoding="Latin-1")



In [4]:
celeb = data['Do you celebrate Thanksgiving?']
print(celeb.value_counts())

does_celebrate = data[data['Do you celebrate Thanksgiving?'] == "Yes"]



Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64


In [4]:
does_celebrate["What is typically the main dish at your Thanksgiving dinner?"].value_counts()

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

In [5]:
tofurkey = does_celebrate[does_celebrate["What is typically the main dish at your Thanksgiving dinner?"]== "Tofurkey"]
print(tofurkey["Do you typically have gravy?"].value_counts())

Yes    12
No      8
Name: Do you typically have gravy?, dtype: int64


In [6]:
apple_isnull = pd.isnull(does_celebrate["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple"])
pumpkin_isnull = pd.isnull(does_celebrate["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin"])
pecan_isnull = pd.isnull(does_celebrate["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan"])

ate_pies = apple_isnull & pumpkin_isnull & pecan_isnull
ate_pies.value_counts()


False    876
True     104
dtype: int64

The cell above shows that out of the 980 people that celebrate thanksgiving, 876 ate either apple, pumpkin, or pecan pie.


In [9]:
def converter(row):
    age = row["Age"]
    if pd.isnull(age) == True:
        return None
    else:
        split_age = age.split(" ")[0]
        num_age = int(split_age[:2])
        return num_age
    


ages = does_celebrate.apply(converter, axis=1)
does_celebrate.insert(loc=len(does_celebrate.columns), column="int_age", value=ages)
print(does_celebrate["int_age"].describe())



        
        
    


count    947.000000
mean      40.089757
std       15.352014
min       18.000000
25%       30.000000
50%       45.000000
75%       60.000000
max       60.000000
Name: int_age, dtype: float64


This cell cuts the age of each respondent to the beginning of their age range(ie for someone in 18-29, their age listed as 18) and looks at some descriptions about the responders age. Since this makes each person the youngest of their range, the description will be skewed to look younger.

In [13]:
def income_converter(person):
    income = person["How much total combined money did all members of your HOUSEHOLD earn last year?"]
    if pd.isnull(income) == True:
        return None
    else:
        first = income.split(" ")[0]
        if first == "Prefer":
            return None
        else:
            raw_number = int(first.replace("$", "").replace(",",""))
            return raw_number
        
        
    
new = does_celebrate.apply(income_converter,axis=1)
does_celebrate.insert(loc=len(does_celebrate.columns),column="int_income",value=new)
print(does_celebrate["int_income"].describe())
            

count       829.000000
mean      75965.018094
std       59068.636748
min           0.000000
25%       25000.000000
50%       75000.000000
75%      100000.000000
max      200000.000000
Name: int_income, dtype: float64


In [54]:
less_than = does_celebrate.loc[does_celebrate["int_income"]<150000]
print(less_than["How far will you travel for Thanksgiving?"].value_counts(normalize=True))

more_than = less_than = does_celebrate.loc[does_celebrate["int_income"]>150000]
print(more_than["How far will you travel for Thanksgiving?"].value_counts(normalize=True))

Thanksgiving is happening at my home--I won't travel at all                         0.407837
Thanksgiving is local--it will take place in the town I live in                     0.294630
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    0.217707
Thanksgiving is out of town and far away--I have to drive several hours or fly      0.079826
Name: How far will you travel for Thanksgiving?, dtype: float64
Thanksgiving is happening at my home--I won't travel at all                         0.480392
Thanksgiving is local--it will take place in the town I live in                     0.245098
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    0.156863
Thanksgiving is out of town and far away--I have to drive several hours or fly      0.117647
Name: How far will you travel for Thanksgiving?, dtype: float64


This data shows that about for people that make less than 150K, 7% more of those people travel compared to those that earn more than 150K.

In [16]:
thanksgiving_pivot = pd.pivot_table(does_celebrate,index="Have you ever tried to meet up with hometown friends on Thanksgiving night?",
                                    columns='Have you ever attended a "Friendsgiving?"', values="int_age")
print(thanksgiving_pivot)

average_income = pd.pivot_table(does_celebrate,index='Have you ever tried to meet up with hometown friends on Thanksgiving night?',
                                columns='Have you ever attended a "Friendsgiving?"', values='int_income')
print(average_income)

Have you ever attended a "Friendsgiving?"                  No        Yes
Have you ever tried to meet up with hometown fr...                      
No                                                  42.283702  37.010526
Yes                                                 41.475410  33.976744
Have you ever attended a "Friendsgiving?"                     No           Yes
Have you ever tried to meet up with hometown fr...                            
No                                                  78914.549654  72894.736842
Yes                                                 78750.000000  66019.736842
